## preprocess.ipynb (multi-k) - Last Pass 기반 학습/테스트 피처 생성

이 노트북은 다음을 수행
1) train/test 이벤트 로드
2) 각 game_episode에서 "마지막 Pass"를 타깃 샘플로 선택
3) 그 마지막 Pass 샘플에 대해 start 기준 공간 피처 + 과거 k개의 이벤트(prev1~prevk) 피처 생성
4) k_prev = {3,5,7,10} 등 여러 버전으로 feature parquet를 저장  
  
**입력**
- data/train.csv : train 이벤트(라벨 end_x/end_y 포함)
- data/test.csv  : test 인덱스 (game_id, game_episode, path)
- data/test/.../{game_episode}.csv : test 이벤트 파일들  
  
**출력**
- artifacts/features_train_k{K}.parquet
- artifacts/labels_train_k{K}.parquet
- artifacts/features_test_k{K}.parquet
- artifacts/test_index.csv

In [1]:
import os
import numpy as np
import pandas as pd

In [2]:
# 경로 설정
DATA_DIR = "data"                   # 원본 데이터 폴더
ART_DIR  = "artifacts"              # 생성한 피처/라벨 저장 폴더
os.makedirs(ART_DIR, exist_ok=True)

# 경기장 상수(좌표 기반 피처 계산에 사용)
PITCH_X, PITCH_Y = 105.0, 68.0
GOAL_X, GOAL_Y   = 105.0, 34.0      # 상대 골대 중앙(보통 x=105, y=34)

print("DATA_DIR:", os.path.abspath(DATA_DIR))
print("ART_DIR :", os.path.abspath(ART_DIR))


DATA_DIR: d:\공모전\스포츠\data
ART_DIR : d:\공모전\스포츠\artifacts


In [3]:
# 1) test.csv(index)에서 실제 이벤트 파일들을 찾아 읽어오는 함수
def load_test_events_from_index(test_index_path: str, base_dir: str = "."):
    """
    test.csv(index 파일)는 "각 game_episode 이벤트가 어디 파일에 있는지(path)"를 갖고 있음.
    이 함수를 통해:
    - test_index(csv) 로드
    - 각 path에 해당하는 이벤트 csv를 읽어서
    - 모든 이벤트를 하나의 DataFrame(test_events)로 합쳐 반환

    Parameters
    ----------
    test_index_path : str
        예: data/test.csv (index 형태)
        columns: game_id, game_episode, path
    base_dir : str
        path가 상대경로일 때 기준이 되는 루트 폴더
        예: base_dir="data"면 path가 "test/xxx.csv"일 때 "data/test/xxx.csv"로 결합

    Returns
    -------
    test_events : pd.DataFrame
        모든 테스트 이벤트를 concat한 DataFrame
    idx : pd.DataFrame
        test_index 원본(정렬/제출 순서 확인용)
    """
    idx = pd.read_csv(test_index_path)

    # test_index에 필수 컬럼이 있는지 검증
    required = {"game_id", "game_episode", "path"}
    missing = required - set(idx.columns)
    if missing:
        raise ValueError(f"test index에 필요한 컬럼이 없습니다: {missing}")

    # 각 path에 해당하는 이벤트 파일들을 읽어 리스트로 모은 뒤 concat
    dfs = []
    for p in idx["path"].tolist():
        full_path = os.path.join(base_dir, p)
        if not os.path.exists(full_path):
            raise FileNotFoundError(f"파일이 없습니다: {full_path}")
        dfs.append(pd.read_csv(full_path))

    test_events = pd.concat(dfs, ignore_index=True)
    return test_events, idx

In [4]:
# 2) start 좌표(start_x, start_y) 기반 공간(geometry) 피처 추가
def add_spatial_from_start(df: pd.DataFrame) -> pd.DataFrame:
    """
    각 이벤트의 시작 위치(start_x, start_y)로부터 파생되는 공간 피처 생성.

    생성되는 피처(예시):
    - start_dist_to_goal      : 시작점에서 골대 중앙까지 거리
    - start_angle_to_goal     : 시작점에서 골대 중앙을 바라보는 각도(atan2)
    - start_dist_to_sideline  : 사이드라인까지의 최소거리 (y 기준)
    - start_dist_to_endline   : 엔드라인(골라인)까지의 거리 (x=105 기준)
    - start_x_ratio           : start_x를 경기장 길이로 정규화(0~1)
    - start_y_centered_abs    : 중앙선(y=34)에서 얼마나 벗어났는지 절댓값
    """
    df = df.copy()

    # 골대까지 x방향 거리 (0이 되면 각도/비율에서 문제가 생길 수 있어 작은 값으로 클립)
    dxg = (GOAL_X - df["start_x"]).clip(lower=1e-6)

    # 중앙선 대비 y방향 거리(절댓값)
    dyg = (df["start_y"] - GOAL_Y).abs()

    # 유클리드 거리: sqrt(dx^2 + dy^2)
    df["start_dist_to_goal"] = np.sqrt(dxg**2 + dyg**2)

    # 각도: arctan2(dy, dx)
    df["start_angle_to_goal"] = np.arctan2(dyg, dxg)

    # 사이드라인까지 최소 거리: min(y, 68 - y)
    df["start_dist_to_sideline"] = np.minimum(df["start_y"], PITCH_Y - df["start_y"])

    # 엔드라인까지 거리: 105 - x
    df["start_dist_to_endline"] = PITCH_X - df["start_x"]

    # 정규화 비율
    df["start_x_ratio"] = df["start_x"] / PITCH_X

    # 중앙선(y=34)에서 벗어난 정도
    df["start_y_centered_abs"] = (df["start_y"] - GOAL_Y).abs()
    return df

In [5]:
# 3) 핵심: 마지막 Pass 샘플을 타깃으로 만들고, 과거 k개 이벤트 피처(prev*)를 붙이기
def build_last_pass_dataset_safe(events: pd.DataFrame, k_prev: int = 5):
    """
    한 경기(episode) 안에는 여러 이벤트가 존재함.
    여기서는 각 game_episode에 대해:
    - type_name == 'Pass' 인 이벤트 중 "마지막 Pass"를 찾고
    - 그 마지막 Pass 한 개를 '타깃 샘플'로 삼아,
      (1) start 기반 공간 피처
      (2) 이전 k개 이벤트의 정보(prev1~prevk)
      (3) episode 단위 요약 피처(ep_*)
    를 생성한다.

    중요한 포인트:
    - shift(i)를 쓰기 때문에 "과거 이벤트가 부족하면" prev 컬럼들은 NaN이 됨.
    - y(end_x, end_y)는 train에서는 존재하지만 test에서는 없으므로
      test에서는 y=None 으로 반환하게 처리.

    Returns
    -------
    X : pd.DataFrame
        학습/예측에 쓸 feature 테이블 (game_episode + feature_cols)
        여기서 row는 "각 episode의 마지막 pass" 1개
    y : pd.DataFrame or None
        train이면 (game_episode, end_x, end_y)
        test이면 None
    """
    df = events.copy()

    # -------------------------
    # (A) 정렬
    # -------------------------
    # episode 내부에서 시간 순서를 정확히 맞추는 게 매우 중요함.
    # 정렬 기준은 (episode, period, time_seconds, action_id).
    df = df.sort_values(
        ["game_episode", "period_id", "time_seconds", "action_id"]
    ).reset_index(drop=True)

    # -------------------------
    # (B) start 기반 공간 피처 추가
    # -------------------------
    df = add_spatial_from_start(df)

    # -------------------------
    # (C) "마지막 Pass"를 타깃으로 지정
    # -------------------------
    # pass 이벤트만 골라 episode별 마지막 row index를 찾음
    is_pass = (df["type_name"] == "Pass")
    if not is_pass.any():
        raise ValueError("type_name=='Pass'가 하나도 없습니다.")

    # episode별 pass 이벤트 중 마지막 1개
    last_pass_idx = df[is_pass].groupby("game_episode", sort=False).tail(1).index

    # 타깃 플래그
    df["is_target_pass"] = False
    df.loc[last_pass_idx, "is_target_pass"] = True

    # episode 그룹 객체(shift, transform에 사용)
    g = df.groupby("game_episode", sort=False)

    # -------------------------
    # (D) 이전 k개 이벤트(prev1~prevk) 피처 생성
    # -------------------------
    # base_cols는 "이전 이벤트에서 가져올 기본 컬럼들"
    base_cols = ["type_name", "result_name", "start_x", "start_y", "end_x", "end_y", "time_seconds"]

    for i in range(1, k_prev + 1):
        # prev{i}_컬럼 = episode 내에서 i만큼 과거 이벤트의 값을 shift로 가져옴
        for c in base_cols:
            df[f"prev{i}_{c}"] = g[c].shift(i)

        # 과거 이벤트 이동량(끝-시작)
        df[f"prev{i}_dx"] = df[f"prev{i}_end_x"] - df[f"prev{i}_start_x"]
        df[f"prev{i}_dy"] = df[f"prev{i}_end_y"] - df[f"prev{i}_start_y"]

        # 과거 이벤트 이동거리
        df[f"prev{i}_dist_move"] = np.sqrt(df[f"prev{i}_dx"]**2 + df[f"prev{i}_dy"]**2)

    # -------------------------
    # (E) prevk 요약 피처(과거 k개를 요약)
    # -------------------------
    prev_dx_cols = [f"prev{i}_dx" for i in range(1, k_prev + 1)]
    prev_dy_cols = [f"prev{i}_dy" for i in range(1, k_prev + 1)]

    # 과거 k개 x방향 이동량 합
    df["prevk_sum_dx"] = df[prev_dx_cols].sum(axis=1, skipna=True)

    # 과거 k개 y방향 이동량 절댓값 합(좌우로 얼마나 흔들렸는지)
    df["prevk_sum_abs_dy"] = df[prev_dy_cols].abs().sum(axis=1, skipna=True)

    # 평균 dx (k_prev로 나눔: 과거 이벤트 수가 부족하면 NaN이 섞이지만 sum(skipna)라 대충 완화됨)
    df["prevk_mean_dx"] = df["prevk_sum_dx"] / max(k_prev, 1)

    # lateral ratio: 좌우 흔들림(절댓값 dy 합) / 전진량(|dx| + eps)
    df["prevk_lateral_ratio"] = df["prevk_sum_abs_dy"] / (df["prevk_sum_dx"].abs() + 1e-6)

    # -------------------------
    # (F) episode 단위 요약 피처(ep_*)
    # -------------------------
    # 이벤트 자체 이동량(현재 이벤트 기준)
    df["dx_evt"] = df["end_x"] - df["start_x"]
    df["dy_evt"] = df["end_y"] - df["start_y"]

    # 마지막 pass(타깃) 이전 이벤트만 집계하기 위한 마스크
    not_target = ~df["is_target_pass"]

    # groupby.transform에서 사용하기 위한 "마스크 적용 합계" 함수
    def nansum_masked(s):
        # 같은 episode index에서 not_target=True인 이벤트만 남기고 합
        arr = s.where(not_target.loc[s.index])
        return np.nansum(arr.values)

    # episode 내에서 타깃 패스 이전 이벤트 개수(= 타깃이 마지막 패스이므로, 사실상 '마지막 패스 이전 이벤트 수')
    df["ep_len_before"] = g["is_target_pass"].transform(lambda x: (~x).sum())

    # episode 시간 범위 (max - min)
    df["ep_time_span"]  = g["time_seconds"].transform(lambda x: x.max() - x.min())

    # 타깃 이전 이벤트들의 dx 합
    df["ep_sum_dx_before"]     = g["dx_evt"].transform(nansum_masked)

    # 타깃 이전 이벤트들의 |dy| 합
    df["ep_sum_abs_dy_before"] = g["dy_evt"].transform(lambda s: nansum_masked(s.abs()))

    # -------------------------
    # (G) 마지막 pass 샘플만 추출 (각 episode당 1개 row)
    # -------------------------
    last_pass = df.loc[df["is_target_pass"]].copy()

    # -------------------------
    # (H) 최종 feature 컬럼 목록 구성
    # -------------------------
    feature_cols = [
        # 식별/메타
        "game_id", "period_id", "time_seconds", "team_id", "player_id", "is_home",

        # 타깃 pass의 시작점
        "start_x", "start_y",

        # start 기반 공간 피처
        "start_dist_to_goal", "start_angle_to_goal", "start_dist_to_sideline", "start_dist_to_endline",
        "start_x_ratio", "start_y_centered_abs",

        # episode 요약 피처
        "ep_len_before", "ep_time_span", "ep_sum_dx_before", "ep_sum_abs_dy_before",

        # prevk 요약 피처
        "prevk_sum_dx", "prevk_sum_abs_dy", "prevk_mean_dx", "prevk_lateral_ratio",

        # 결과(성공/실패 등) - 분기 모델에 활용 가능
        "result_name",
    ]

    # 과거 k개 이벤트의 상세 피처를 모두 추가
    for i in range(1, k_prev + 1):
        feature_cols += [
            f"prev{i}_type_name",
            f"prev{i}_result_name",
            f"prev{i}_start_x", f"prev{i}_start_y",
            f"prev{i}_end_x",   f"prev{i}_end_y",
            f"prev{i}_dx",      f"prev{i}_dy", f"prev{i}_dist_move",
            f"prev{i}_time_seconds",
        ]

    # -------------------------
    # (I) X(피처 테이블) 만들기
    # -------------------------
    # game_episode는 학습/제출 키로 쓰이므로 항상 포함
    X = last_pass[["game_episode"] + feature_cols].copy()

    # -------------------------
    # (J) y(라벨) 만들기: train에서만 존재
    # -------------------------
    y = None
    # end_x/end_y가 존재하고 결측이 없으면 y 생성
    if {"end_x", "end_y"}.issubset(last_pass.columns) and last_pass[["end_x", "end_y"]].notna().all().all():
        y = last_pass[["game_episode", "end_x", "end_y"]].copy()

    return X, y

#### 메인 실행: train/test 로드 → test_index 저장 → K_LIST별 피처 저장

In [6]:
# 1) Load events
# train.csv: 이벤트 + 라벨(end_x/end_y)이 있는 것으로 가정
train_events = pd.read_csv(os.path.join(DATA_DIR, "train.csv"))

# test.csv는 "각 episode 이벤트 파일의 경로"를 담고 있으므로,
# load_test_events_from_index로 실제 이벤트를 합쳐서 test_events를 만든다.
test_events, test_index = load_test_events_from_index(
    os.path.join(DATA_DIR, "test.csv"),
    base_dir="data"
)

print("train_events:", train_events.shape)
print("test_events :", test_events.shape)
print("test_index  :", test_index.shape)

# 제출 순서/정렬 확인용으로 test_index 저장
test_index.to_csv(os.path.join(ART_DIR, "test_index.csv"), index=False)

train_events: (356721, 15)
test_events : (53110, 15)
test_index  : (2414, 3)


In [7]:
# 2) Build & save multiple k_prev versions
K_LIST = [3, 5, 7, 10]

for K_PREV in K_LIST:
    print(f"\n=== Building features for k_prev={K_PREV} ===")

    # train용: 마지막 pass 샘플 + 라벨 생성
    X_train, y_train = build_last_pass_dataset_safe(train_events, k_prev=K_PREV)

    # test용: 마지막 pass 샘플 + (라벨 없음 -> y는 None)
    X_test, _        = build_last_pass_dataset_safe(test_events,  k_prev=K_PREV)

    # train 라벨이 반드시 있어야 함(없으면 학습 불가)
    assert y_train is not None, f"y_train이 None입니다(k_prev={K_PREV}). train 라벨을 확인하세요."

    # 저장 경로
    train_feat_path  = os.path.join(ART_DIR, f"features_train_k{K_PREV}.parquet")
    train_label_path = os.path.join(ART_DIR, f"labels_train_k{K_PREV}.parquet")
    test_feat_path   = os.path.join(ART_DIR, f"features_test_k{K_PREV}.parquet")

    # parquet 저장 (index=False: 불필요한 인덱스 저장 방지)
    X_train.to_parquet(train_feat_path, index=False)
    y_train.to_parquet(train_label_path, index=False)
    X_test.to_parquet(test_feat_path, index=False)

    print("Saved:")
    print(" -", train_feat_path)
    print(" -", train_label_path)
    print(" -", test_feat_path)


=== Building features for k_prev=3 ===
Saved:
 - artifacts\features_train_k3.parquet
 - artifacts\labels_train_k3.parquet
 - artifacts\features_test_k3.parquet

=== Building features for k_prev=5 ===
Saved:
 - artifacts\features_train_k5.parquet
 - artifacts\labels_train_k5.parquet
 - artifacts\features_test_k5.parquet

=== Building features for k_prev=7 ===
Saved:
 - artifacts\features_train_k7.parquet
 - artifacts\labels_train_k7.parquet
 - artifacts\features_test_k7.parquet

=== Building features for k_prev=10 ===


C:\Users\cho03\AppData\Local\Temp\ipykernel_392\474063207.py:77: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  df[f"prev{i}_dist_move"] = np.sqrt(df[f"prev{i}_dx"]**2 + df[f"prev{i}_dy"]**2)
C:\Users\cho03\AppData\Local\Temp\ipykernel_392\474063207.py:70: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  df[f"prev{i}_{c}"] = g[c].shift(i)
C:\Users\cho03\AppData\Local\Temp\ipykernel_392\474063207.py:70: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor

Saved:
 - artifacts\features_train_k10.parquet
 - artifacts\labels_train_k10.parquet
 - artifacts\features_test_k10.parquet


C:\Users\cho03\AppData\Local\Temp\ipykernel_392\474063207.py:123: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  df["ep_sum_abs_dy_before"] = g["dy_evt"].transform(lambda s: nansum_masked(s.abs()))
